<h1>Neighborhoods in Toronto</h1>
<p>I am going to explore and cluster the neighborhoods in Toronto based on wikipedia and foursquere data.</p>

<h3>Create dataframe</h3>

Firstly, I downloaded the table and created csv file. Let's read data into pandas dataframe.

In [ ]:
import pandas as pd
codes=pd.read_csv("postal_codes.csv",sep=";")
codes.head()

<h3>Pre-processing</h3>

Removing cells with a borough that is Not assigned.

In [ ]:
codes.shape

In [ ]:
import numpy as np
codes=codes.drop(np.where(codes["Borough"]=="Not assigned")[0])
codes.shape

<h3>Adding coordinates</h3>

Installing pgeocode

In [ ]:
! pip install pgeocode

In [ ]:
import pgeocode
nomi = pgeocode.Nominatim('ca')

#Func for getting latitude
def get_lat(code):
    return nomi.query_postal_code(code).latitude

#Func for getting longitude
def get_lon(code):
    return nomi.query_postal_code(code).longitude

codes["Latitude"]=codes["Postal Code"].apply(get_lat)
codes["Longitude"]=codes["Postal Code"].apply(get_lon)

codes.head()

Check our data

In [ ]:
codes.isna().sum()

One code has not been found in pgeocod. Lets delete it

In [ ]:
codes.dropna(axis=0,inplace=True)
codes.shape

<h3>Creating a map of Toronto</h3>

Get Toronto coordinates

In [ ]:
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
import folium

# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(codes['Latitude'], codes['Longitude'], codes['Borough'], codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3>Downtown Toronto - preprocessing data</h3>

I decided to explore Downtown Toronto. Firstly, I create the another dataframe for it.

In [ ]:
downtown_df = codes[codes['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()

Get Downtown coordinates

In [ ]:
address = 'Toronto, Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

Creating the map of Toronto Downtown

In [ ]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'],downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

<h3>Explore Neighborhoods in Toronto Downtown</h3>

I use the Foursquare API to explore the neighborhoods and segment them.

In [ ]:
import requests # library to handle requests

CLIENT_ID = '44TCPS2WRSRAMAYRI3K1AEA2SAZ1EIDR3BUQX5RSVSFSYE13' 
CLIENT_SECRET = 'BOOEV2DB1NV2ORS3BQ101Z4QV1NQB0YNAYE1AR1PCT3NEQXJ' 
VERSION = '20180605' 

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Function for getting venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create the dataframe for Toronto Downtown using my function

In [ ]:
downtown_venues = getNearbyVenues(names=downtown_df['Neighborhood'],
                                  latitudes=downtown_df['Latitude'],
                                  longitudes=downtown_df['Longitude']
                                  )

Let's check the size of the resulting dataframe

In [ ]:
print(downtown_venues.shape)
downtown_venues.head()

Let's check how many venues were returned for each neighborhood

In [ ]:
downtown_venues.groupby('Neighborhood').count()

Find out how many unique categories can be curated from all the returned venues

In [ ]:
len(downtown_venues['Venue Category'].unique())

<h3>Analyze Each Neighborhood<h3>

In [ ]:
# one hot encoding
downtn_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtn_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

downtn_onehot.head()

Examine the dataframe size.

In [ ]:
downtn_onehot.shape

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
downtn_grouped = downtn_onehot.groupby('Neighborhood').mean().reset_index()
downtn_grouped

Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in downtn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtn_grouped[downtn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

A function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtn_grouped['Neighborhood']

for ind in np.arange(downtn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<h3>Cluster Neighborhoods</h3>

Run k-means to cluster the neighborhood into 3 clusters.

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

downtn_grouped_clustering = downtn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtn_merged = downtown_df
downtn_merged = downtn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtn_merged.head() 

Finally, let's visualize the resulting clusters

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtn_merged['Latitude'], downtn_merged['Longitude'], downtn_merged['Neighborhood'], downtn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>

Cluster 1 - residential areas

In [ ]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 0, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

Cluster 2 - harbor

In [ ]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 1, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

Cluster 3 - business center

In [ ]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 2, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]